In [26]:
import os
import cv2
from paddleocr import PPStructure,draw_structure_result,save_structure_res
from PIL import Image
import save_pic
from recognition import recognize
# def savePic(a,path):
#     for i in range(len(a)):
#         if os.path.exists("./input_images/"):
#             pass
#         else:
#              os.mkdir("./input_images/")
#         imgPath = "./input_images/" + str(i+1) + ".jpg"  #notes: 图片的扩展名要一致
#         im = Image.open(path)
#         im = im.crop((a[i][0],a[i][1],a[i][2],a[i][3]))  # 对图片进行切割 im.crop(top_x, top_y, bottom_x, bottom_y)
#         im.save(imgPath)

def BanMian(img_path,save_folder='./output1'):
    table_engine = PPStructure(show_log=True,layout_model_dir='https://paddleocr.bj.bcebos.com/ppstructure/models/layout/picodet_lcnet_x1_0_fgd_layout_table_infer.tar',download=True)

    #table_engine = PPStructure(show_log=True,layout_model_dir='https://paddleocr.bj.bcebos.com/ppstructure/models/layout/picodet_lcnet_x1_0_fgd_layout_cdla_infer.tar',download=True)
    #table_engine = PPStructure(show_log=True,layout_model_dir='https://paddleocr.bj.bcebos.com/ppstructure/models/layout/picodet_lcnet_x1_0_fgd_layout_infer.tar',download=True)

    # save_folder = './output'
    # img_path = './origin_img/22.jpg'
    img = cv2.imread(img_path)
    result = table_engine(img)
    save_structure_res(result, save_folder,os.path.basename(img_path).split('.')[0])
    # for line in result:
    #     line.pop('img')
    #     print(line)
    a=[]
    for res in result:
        print(res)
        if res['type']== 'text':
            a.append(res['bbox'])
    # savePic(a,img_path)
    # location=save_pic.save(a)
    # res=recognize(a)
    # dic_txt_loc=[]
    # for i in range(len(location)):
    #     aa=location[i]
    #     txts=res[i]
    #     for j in range(len(aa)):
    #         dic_txt_loc.append([txts[j],aa[j]])
    return a
    # return dic_txt_loc

In [27]:
import cv2
import os
from itertools import product
from PIL import Image
def remove_shuiyin(file_path):
    img = Image.open(file_path)
    width, height = img.size
    for pos in product(range(width), range(height)):
        if sum(img.getpixel(pos)[:3]) > 500:
            img.putpixel(pos, (255, 255, 255))
    img.save(file_path)

In [28]:
res=[]
for i in range(1,8):
    remove_shuiyin('./img/{}.jpg'.format(str(i)))
    res0=BanMian('./img/{}.jpg'.format(str(i)))
    res.append(res0)

[2023/05/27 15:20:07] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir=None, cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\97885/.paddleocr/whl\\det\\ch\\ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, download=True, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set='totaltext', enable_mkl

In [29]:
res

[[[176, 373, 1310, 1534]],
 [[165, 487, 1934, 713], [163, 906, 1935, 1296]],
 [[172, 177, 1934, 1044]],
 [[168, 1093, 1933, 1282], [167, 216, 1937, 896]],
 [[171, 177, 1932, 1187]],
 [[167, 348, 1935, 1275]],
 [[171, 178, 1930, 628]]]

In [30]:
for i in range(len(res)):
    res[i].sort(key=lambda x:(x[1]+x[0]))
##这一步很关键

In [31]:
if os.path.exists('./part_table'):
    pass
else:
    os.mkdir('./part_table')
def cut_table(aa):
    for i in range(len(aa)):
        for j in range(len(aa[i])):
            if os.path.exists('./part_table/'+str(i+1)):
                pass
            else:
                os.mkdir('./part_table/'+str(i+1))
            SavePath='./part_table/'+str(i+1)+'/'+str(j+1)+'.jpg'
            im = Image.open('./img/'+str(i+1)+'.jpg')
            im = im.crop((aa[i][j][0],aa[i][j][1],aa[i][j][2],aa[i][j][3]))  # 对图片进行切割 im.crop(top_x, top_y, bottom_x, bottom_y)
            im.save(SavePath)

In [32]:
cut_table(res)

In [33]:
from paddleocr import PaddleOCR, draw_ocr

def shibie(name):
    ocr = PaddleOCR(use_angle_cls=True, lang="ch")  # need to run only once to download and load model into memory
    img_path = name
    result = ocr.ocr(img_path, cls=True)
    # for idx in range(len(result)):
    #     res = result[idx]
    #     for line in res:
    #         print(line)

    # 显示结果
    # 如果本地没有simfang.ttf，可以在doc/fonts目录下下载
    from PIL import Image
    result = result[0]
    image = Image.open(img_path).convert('RGB')
    boxes = [line[0] for line in result]
    txts = [line[1][0] for line in result]
    print(txts)
    scores = [line[1][1] for line in result]
    allstring=""
    for i in txts:
        allstring+=i
    return allstring

In [34]:
res

[[[176, 373, 1310, 1534]],
 [[165, 487, 1934, 713], [163, 906, 1935, 1296]],
 [[172, 177, 1934, 1044]],
 [[167, 216, 1937, 896], [168, 1093, 1933, 1282]],
 [[171, 177, 1932, 1187]],
 [[167, 348, 1935, 1275]],
 [[171, 178, 1930, 628]]]

In [35]:
ans_char=[]
num=0
pin_num=1
page_lian=[]
for i in range(len(res)):
    for j in range(len(res[i])):
        if j==len(res[i])-1 and i< len(res)-1:
            this_pic=res[i][j]
            next_pic=res[i+1][0]
            ##最后一张表格的下边界开始裁剪，如果剩余的部分除了页码还有其他的文字，就说明肯定不会链接在一起
            im_this=Image.open('./img/{}.jpg'.format(str(i+1)))
            this_remain_pic=im_this.crop((this_pic[0],this_pic[3],im_this.size[0],im_this.size[1]))
            this_remain_pic.save('test.jpg')
            this_char=shibie('test.jpg')
            if len(this_char)>1:
                print("说明剩下的部分还有文字，说明下一页肯定不是切断的表格")
                now=Image.open('./part_table/'+str(i+1)+'/{}.jpg'.format(j+1))
                now.save('./pinjie/{}.jpg'.format(pin_num))
                pin_num+=1
            else:
                #这一页的表格是页面的末尾，这个时候就需要考虑下一页的头部了
                im_next=Image.open('./img/{}.jpg'.format(str(i+2)))
                next_remain_pic=im_next.crop((0,0,next_pic[3],next_pic[1]))
                next_remain_pic.save('test.jpg')
                next_char=shibie('test.jpg')
                if len(next_char) >=2:
                    now=Image.open('./part_table/'+str(i+1)+'/{}.jpg'.format(j+1))
                    now.save('./pinjie/{}.jpg'.format(pin_num))
                    pin_num+=1
                    print('说明下一页肯定不是切断的表格，因为之前还有文字')
                else:
                    #这样的话，多半就能判断下一页和上一页是同一个表格了，因此可以将二者合并
                    #那要如何合并呢？暂时先不管共同表头合并试试，之后再删除共同表头
                    page_lian.append(i+1)
                    this_pic_old_path='./part_table/'+str(i+1)+'/{}.jpg'.format(str(j+1))
                    next_pic_old_path='./part_table/'+str(i+2)+'/1.jpg'
                    img1 = Image.open(this_pic_old_path)
                    img2 = Image.open(next_pic_old_path)
                    result = Image.new(img1.mode, (img1.size[0],img1.size[1]+img2.size[1]))
                    result.paste(img1, box=(0, 0))
                    result.paste(img2, box=(0,img1.size[1]))
                    result.save("./pinjie/{}.jpg".format(str(pin_num)))
                    pin_num+=1

[2023/05/27 15:23:06] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\97885/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\97885/.paddleocr/whl\\det\\ch\\ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thre

In [36]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
table_recognition = pipeline(Tasks.table_recognition, model='damo/cv_dla34_table-structure-recognition_cycle-centernet')

2023-05-27 15:24:24,468 - modelscope - INFO - Model revision not specified, use the latest revision: v0.1
2023-05-27 15:24:25,751 - modelscope - INFO - initiate model from C:\Users\97885\.cache\modelscope\hub\damo\cv_dla34_table-structure-recognition_cycle-centernet
2023-05-27 15:24:25,751 - modelscope - INFO - initiate model from location C:\Users\97885\.cache\modelscope\hub\damo\cv_dla34_table-structure-recognition_cycle-centernet.
2023-05-27 15:24:25,767 - modelscope - WARNING - No preprocessor field found in cfg.
2023-05-27 15:24:25,767 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-05-27 15:24:25,767 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'C:\\Users\\97885\\.cache\\modelscope\\hub\\damo\\cv_dla34_table-structure-recognition_cycle-centernet'}. trying to build by task and model information.
2023-05-27 15:24:25,767 - modelscope - WARNIN

In [37]:
cut_table(res)
location1=[]
location2=[]
pin_num=1
for i in range(len(res)):
    for j in range(len(res[i])):
        path='./part_table/{}/'.format(str(i+1))+'{}.jpg'.format(str(j+1))

        if  j==len(res[i])-1 and page_lian.count(i+1)!=0:

            next_path='./part_table/{}/1.jpg'.format(str(i+2))

            if page_lian.count(i)!=0 and len(res[i])==1:
                img1=Image.open(path)
                img2=Image.open(next_path)
                result = Image.new(img1.mode, (img1.size[0],img1.size[1]+img2.size[1]))
                result.paste(img1, box=(0, 0))
                result.paste(img2, box=(0,img1.size[1]))
                if len(res[i + 1])==1:
                    result.save(next_path)
                else:
                    result.save("./pinjie_removehead/{}.jpg".format(str(pin_num)))
                    pin_num+=1
                continue

            result1 = table_recognition(path)
            #就说明，这一张表后面还跟着一张表
            result2 = table_recognition(next_path)
            #然后对两个表识别出来的结果，如果有公共表头，则删去第二张表的公共表头
            #这里还没有写完，由于modelscope效果不好
            after_res=[]
            for k in result1['polygons']:
                #print(k)
                if abs(k[0]-k[4])<10:
                    continue
                temp=[int(k[0]),int(k[1]),int(k[4]),int(k[5])]
                after_res.append(temp)
            location1.append(after_res)
            after_res=[]
            for k in result2['polygons']:
                #print(k)
                if abs(k[0]-k[4])<10:
                    continue
                temp=[int(k[0]),int(k[1]),int(k[4]),int(k[5])]
                after_res.append(temp)
            location2.append(after_res)
            location1[0].sort(key=lambda x:(x[1]+x[0]))
            location2[0].sort(key=lambda x:(x[1]+x[0]))
            ##这里非常的美妙，按照xy之和排序，因为会有一到二的偏差，如果按x再按y排序的话，还会有几率后面的到前面来

            res1_first=location1[0][0]
            res2_first=location2[0][0]
            img1=Image.open(path)
            img2=Image.open(next_path)
            img1_first=img1.crop(res1_first)
            img2_first=img2.crop(res2_first)
            img1_first.save("./temp1.jpg")
            img2_first.save("./temp2.jpg")
            string1=shibie("./temp1.jpg")
            string2=shibie("./temp2.jpg")
            os.remove("./temp1.jpg")
            os.remove("./temp2.jpg")
            if string1!="" and string2==string1:
                ##这个时候两个表头就是相同的
                print(string1,string2)
                print("yes!")
                ##开始裁剪第二张表格的表头
                img2_cut=img2.crop((res2_first[0],res2_first[3],img2.size[0],img2.size[1]))
                #img2_cut.save('./test.jpg')
                result = Image.new(img1.mode, (img1.size[0],img1.size[1]+img2_cut.size[1]))
                result.paste(img1, box=(0, 0))
                result.paste(img2_cut, box=(0,img1.size[1]))
            else:
                ##这个时候表格是不相同的，因此直接拼接即可
                result = Image.new(img1.mode, (img1.size[0],img1.size[1]+img2.size[1]))
                result.paste(img1, box=(0, 0))
                result.paste(img2, box=(0,img1.size[1]))
            if len(res[i + 1])==1 and page_lian.count(i+2)!=0:
                result.save(next_path)
            else:
                result.save("./pinjie_removehead/{}.jpg".format(str(pin_num)))
                pin_num+=1
        else:
            if j!=0 or page_lian.count(i)==0:
                result1=Image.open(path)
                result1.save("./pinjie_removehead/{}.jpg".format(str(pin_num)))
                pin_num+=1

2023-05-27 15:24:26,675 - modelscope - WARNING - task table-recognition input definition is missing


[2023/05/27 15:24:43] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\97885/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\97885/.paddleocr/whl\\det\\ch\\ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thre

In [60]:
##下面是测试阿里云单元格识别的效果，发现效果不好，因为单元格太复杂

[[[176, 373, 1310, 1534]],
 [[165, 487, 1934, 714], [163, 906, 1935, 1296]],
 [[172, 177, 1933, 1044]],
 [[167, 216, 1937, 896], [168, 1093, 1934, 1282]],
 [[171, 177, 1932, 1187]],
 [[167, 348, 1935, 1275]],
 [[171, 178, 1930, 628]]]

In [26]:
location1[0][0]
location2

[[[66, 3, 188, 108],
  [187, 3, 308, 108],
  [307, 3, 428, 108],
  [428, 3, 606, 108],
  [606, 3, 783, 108],
  [781, 3, 1580, 42],
  [1580, 3, 1754, 108],
  [1, 4, 67, 108],
  [783, 42, 941, 108],
  [942, 42, 1101, 108],
  [1102, 42, 1261, 108],
  [1261, 42, 1420, 108],
  [1421, 42, 1580, 108],
  [3, 108, 68, 188],
  [67, 108, 188, 188],
  [188, 108, 310, 188],
  [308, 108, 428, 188],
  [428, 108, 605, 188],
  [606, 108, 782, 188],
  [783, 108, 940, 188],
  [941, 108, 1101, 188],
  [1101, 108, 1260, 188],
  [1261, 108, 1420, 188],
  [1420, 108, 1579, 188],
  [1580, 108, 1754, 187],
  [2, 188, 68, 267],
  [68, 188, 188, 267],
  [188, 188, 310, 267],
  [310, 188, 428, 267],
  [428, 188, 605, 267],
  [605, 188, 782, 267],
  [782, 188, 940, 267],
  [940, 188, 1101, 267],
  [1101, 188, 1260, 267],
  [1260, 188, 1420, 267],
  [1420, 188, 1579, 267],
  [1579, 188, 1754, 266],
  [68, 267, 188, 347],
  [188, 267, 310, 346],
  [310, 267, 427, 346],
  [428, 267, 605, 347],
  [605, 267, 782, 346],

In [13]:
location=[]
path='./part_table/2/2.jpg'
result = table_recognition(path)
after_res=[]
print(result)
for k in result['polygons']:
    #print(k)
    if abs(k[0]-k[4])<5:
        continue
    temp=[int(k[0]),int(k[1]),int(k[4]),int(k[5])]
    after_res.append(temp)
location.append(after_res)

{'polygons': array([[ 614.5574   ,    6.077574 ,  615.3407   ,  112.75558  ,
         792.88275  ,  113.04138  ,  791.4493   ,    6.301593 ],
       [ 316.72974  ,    6.3138447,  317.04727  ,  111.17699  ,
         438.00092  ,  112.657616 ,  438.01163  ,    6.3396688],
       [ 197.1007   ,    6.2216387,  197.4126   ,  111.244316 ,
         317.04727  ,  111.17699  ,  316.72974  ,    6.3138447],
       [  75.49791  ,    6.2870703,   76.66031  ,  111.13442  ,
         197.4126   ,  111.244316 ,  197.1007   ,    6.2216387],
       [  12.134846 ,  186.09613  ,   11.374354 ,  346.73352  ,
         241.38869  ,  267.6405   ,  241.37798  ,  227.23273  ],
       [ 949.32117  ,  227.4577   ,  949.3145   ,  267.9129   ,
        1109.368    ,  267.99316  , 1109.4198   ,  227.44386  ],
       [ 791.06976  ,  227.67569  ,  791.11456  ,  268.14398  ,
         949.3145   ,  267.9129   ,  949.32117  ,  227.4577   ],
       [ 951.6336   ,   44.7552   ,  952.01776  ,  112.95974  ,
        1111.535    

In [14]:
location[0].sort(key=lambda x:(x[1],x[0]))

In [15]:
len(location[0])

40

In [17]:
from PIL import ImageDraw

# num=1
# for i in range(len(location[0])):
#     im=Image.open(path)
#     img=im.crop((location[0][i][0],location[0][i][1],location[0][i][2],location[0][i][3]))
#     img.save('./temp/{}.jpg'.format(str(num)))
#     num+=1
im=Image.open(path)
a=ImageDraw.ImageDraw(im)
for i in range(len(location[0])):
    a.rectangle(((location[0][i][0],location[0][i][1]),(location[0][i][2],location[0][i][3])),fill=None,outline="red",width=2)
im.save('res.jpg')

In [133]:
location

[[[12, -1, 75, 9],
  [70, 0, 200, 9],
  [10, 8, 75, 111],
  [74, 8, 195, 111],
  [195, 8, 316, 111],
  [316, 8, 437, 111],
  [437, 8, 614, 112],
  [613, 8, 791, 112],
  [1589, 8, 1764, 112],
  [791, 43, 950, 112],
  [1269, 43, 1429, 112],
  [1429, 43, 1588, 111],
  [950, 44, 1110, 112],
  [1109, 44, 1269, 112],
  [10, 185, 242, 348],
  [10, 185, 790, 228],
  [1428, 186, 1589, 227],
  [1589, 186, 1764, 228],
  [790, 187, 948, 228],
  [948, 187, 1108, 228],
  [1108, 187, 1267, 227],
  [1267, 187, 1428, 227],
  [241, 227, 790, 267],
  [1267, 227, 1428, 267],
  [1428, 227, 1589, 267],
  [1589, 227, 1764, 267],
  [790, 228, 948, 267],
  [948, 228, 1108, 267],
  [1108, 228, 1267, 267],
  [241, 267, 790, 307],
  [790, 267, 948, 307],
  [948, 267, 1108, 307],
  [1108, 267, 1267, 307],
  [1267, 267, 1428, 307],
  [1428, 267, 1589, 307],
  [1589, 267, 1764, 307],
  [790, 307, 949, 349],
  [948, 307, 1108, 349],
  [1108, 307, 1267, 348],
  [1267, 307, 1428, 348],
  [1428, 307, 1589, 348],
  [1589